In [1]:
#| default_exp sandbox

In [2]:
from jupyterlab.extensions.pypi import https_proxy_url
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

In [20]:
#| hide
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [18]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [21]:
os.getenv('GC_QUOTE_API_CREDENTIALS')
os.getenv('HTTP_PROXY')
os.getenv('HTTPS_PROXY')

'res/quote-api-365206-39d39b2fde07.json'

'http://127.0.0.1:20171'

'http://127.0.0.1:20171'

In [22]:
from dotenv import dotenv_values
config = dotenv_values("../.env")
key = config.get('GC_QUOTE_API_CREDENTIALS')

In [23]:
from google.cloud import bigquery
import pandas_gbq
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file("../" + key)

In [24]:
client = bigquery.Client(credentials=credentials)

QUERY = """
		select *
		from `quote-api-365206.report.ride_dispatch_v` r
		where dispatch_amount_net_usd is not null
        limit 200
		"""

In [8]:
QUERY_DISINCT = """
		select count (*)
		from `quote-api-365206.report.ride_dispatch_v` r
		where trip_type is null
        limit 200
		"""

In [9]:
!echo $HTTP_PROXY

http://127.0.0.1:20171


In [10]:
#query_job = client.query(QUERY)

In [25]:

df = pandas_gbq.read_gbq(QUERY,credentials=credentials)

Downloading: 100%|██████████|


In [12]:

df_trip_type = pandas_gbq.read_gbq(QUERY_DISINCT,credentials=credentials)

Downloading: 100%|██████████|


In [26]:

df.head()

,ride_id,trip_no,dispatch_id,trip_type,ride_stat,dispatch_stat,partner_tran_id,partner_id,from_fleet_id_as_customer,pickup_date,...,has_complaint,has_fleet_complaint,has_penalty_complaint,has_ops_complaint,gps_ad_check,gps_cns_check,score,incoming_dispatch_info_last_updated_at,dispatch_info_last_updated_at,etl_time
0,2324292,1,345056,None,Accepted,At destination,161955,1,16,2022-08-01,...,0,0,0,0,0,0,None,2022-07-08 20:18:48+00:00,2022-08-01 23:58:01+00:00,2024-08-29 23:05:33.393026+00:00
1,2398953,0,518874,None,Accepted,Customer no show,237411,1,16,2022-11-24,...,1,1,0,0,0,0,None,2022-11-24 09:28:34+00:00,2022-11-24 09:03:50+00:00,2024-08-29 23:05:33.393026+00:00
2,2378500,0,472749,None,Accepted,At destination,216589,1,16,2022-10-21,...,1,0,1,1,0,0,None,2023-01-10 16:48:45+00:00,2023-01-10 01:08:03+00:00,2024-08-29 23:05:33.393026+00:00
3,2382836,0,482760,None,Accepted,Customer no show,221063,1,16,2022-11-25,...,1,1,0,0,0,0,None,2022-12-02 10:25:19+00:00,2022-11-25 12:12:22+00:00,2024-08-29 23:05:33.393026+00:00
4,3316247,0,2340475,point2point,Pending,At destination,1162413,1,16,2024-03-28,...,0,0,0,0,0,0,None,2024-04-18 05:23:39+00:00,2024-04-18 05:24:06+00:00,2024-08-29 23:05:33.393026+00:00


In [14]:
QUERY_ROUTE = """
        SELECT r.*, p1.lat from_lat, p1.lng from_lng, p2.lat to_lat, p2.lng to_lng
        FROM `report.ride_dispatch_v` r
        left JOIN `report.dim_place` p1
        ON r.pickup_place_id=p1.id
        INNER JOIN `report.dim_place` p2
        ON r.pickup_place_id=p2.id
        ORDER BY ride_id DESC
        limit 2;
		"""


In [15]:
df_route = pandas_gbq.read_gbq(QUERY_ROUTE,credentials=credentials)

Downloading: 100%|██████████|


In [16]:
df_route.head()

,ride_id,trip_no,dispatch_id,trip_type,ride_stat,dispatch_stat,partner_tran_id,partner_id,from_fleet_id_as_customer,pickup_date,...,gps_ad_check,gps_cns_check,score,incoming_dispatch_info_last_updated_at,dispatch_info_last_updated_at,etl_time,from_lat,from_lng,to_lat,to_lng
0,3766119,0,3218599,point2point,Pending,None,1584248,2,18,2024-09-03,...,0,0,None,2024-08-29 22:51:31+00:00,NaT,2024-08-29 23:05:33.393026+00:00,9.547790000,100.062000000,9.547790000,100.062000000
1,3766118,0,3218598,point2point,Pending,None,1584247,2,18,2024-09-06,...,0,0,None,2024-08-29 22:51:12+00:00,NaT,2024-08-29 23:05:33.393026+00:00,25.796914000,-80.288455000,25.796914000,-80.288455000


In [17]:
df_route.dtypes

ride_id                                                 Int64
trip_no                                                 Int64
dispatch_id                                             Int64
trip_type                                              object
ride_stat                                              object
dispatch_stat                                          object
partner_tran_id                                         Int64
partner_id                                              Int64
from_fleet_id_as_customer                               Int64
pickup_date                                            dbdate
pickup_time                                            dbtime
pickup_datetime                           datetime64[us, UTC]
pickup_local_datetime                          datetime64[us]
pickup_local_datetime_str                              object
pickup_place_id                                         Int64
dropoff_place_id                                        Int64
passenge